# 1) ETL (b) Extraction

In [ ]:
pip install -r requirements.txt

In [2]:
from settings import GEMINI_API_KEY
from data.ingest import GoogleDriveExtractor

extractor = GoogleDriveExtractor(
    credentials_file="smart-platform.json", 
    sheet_name="Master Database",
    worksheet_name="inventory", 
    gemini_api_key=GEMINI_API_KEY 
)

In [3]:
folder_details_dict = extractor.view_folder_details(folder_link="https://drive.google.com/drive/folders/103epRgaKBSrXuLI7dhnfVLPs4d3bj94o?usp=drive_link",
                              credentials_file = "smart-platform.json")

Folder Name: 01 Malacca (Shaz), Share Link: https://drive.google.com/drive/folders/1nstotWI9LYvUamNw-NSew-jnVDD7VAaH, Item Count: 2
Folder Name: 17 Others, Share Link: https://drive.google.com/drive/folders/1pRG5W1oJEc8Ify3XQ0iWY1NXM-63FzhN, Item Count: 7
Folder Name: 16 Umrah (Norra & Zul), Share Link: https://drive.google.com/drive/folders/1AQW3PViswZN18QRPTAhAdgrt9fxi6xsq, Item Count: 16
Folder Name: 15 Perak (Malena), Share Link: https://drive.google.com/drive/folders/1Ne8NVDgbbvDvn4OKnzW3bQDl6g-y6Llx, Item Count: 3
Folder Name: 14 Perlis (Noey), Share Link: https://drive.google.com/drive/folders/1GlEZiVla7ZQk1u8EcOVCM84nu_hXDwVk, Item Count: 9
Folder Name: 13 Kedah (Noey), Share Link: https://drive.google.com/drive/folders/1eEu1iI23EAVTAjlzFGM3sF5XY1NIAAZF, Item Count: 18
Folder Name: 12 Kelantan (Noey), Share Link: https://drive.google.com/drive/folders/1ohGD8AgMID2IdT3oo3bU_QZQatM6eC0x, Item Count: 1
Folder Name: 11 Sabah (Zul), Share Link: https://drive.google.com/drive/folders

['15 Perak (Malena)',
 '14 Perlis (Noey)',
 '13 Kedah (Noey)',
 '12 Kelantan (Noey)']

In [7]:
# restarted from sabah
# folder_link = folder_details_dict['10 Sarawak (Norra)']+'?usp=drive_link'
for folder_name in list(folder_details_dict.keys())[3:7]:
    print(f"=====> {folder_name}")
    folder_link = folder_details_dict[folder_name]+'?usp=drive_link'
    print(folder_link)
    extractor.run_ETL(folder_link)


=====> 15 Perak (Malena)
https://drive.google.com/drive/folders/1Ne8NVDgbbvDvn4OKnzW3bQDl6g-y6Llx?usp=drive_link
Found 3 PDF files in the specified folder.
1 ---> Processing: partners - perak - accommodation/tingkat valley 
Downloading: partners - perak - accommodation/tingkat valley 
Download 100%
Downloaded: partners - perak - accommodation/tingkat valley 
Performing OCR on: partners - perak - accommodation/tingkat valley 
file_path: partners - perak - accommodation/tingkat valley.pdf --> image_path: partners - perak - accommodation/tingkat valley_page_1.jpg
file_path: partners - perak - accommodation/tingkat valley.pdf --> image_path: partners - perak - accommodation/tingkat valley_page_2.jpg
Updating Google Sheet with: partners - perak - accommodation/tingkat valley 
Data appended to 'Master Database' - 'inventory'
2 ---> Processing: partners - perak - nature/tasik cermin ipoh sdn bhd
Downloading: partners - perak - nature/tasik cermin ipoh sdn bhd
Download 100%
Downloaded: partner

# 1) ETL (c) Transformation

In [1]:
from gdrive.gdrive_handler import GspreadHandler

worksheet_name = "inventory_processed"
worksheet_name = "inventory"
def get_df(sheet_name, worksheet_name):
    gspread_handler = GspreadHandler(credentials_filepath='smart-platform.json')
    df = gspread_handler.get_sheet_as_df(sheet_name=sheet_name, worksheet_name=worksheet_name)
    return df

df_raw = get_df(sheet_name = "Master Database" , worksheet_name="inventory")
df = get_df(sheet_name = "Master Database" , worksheet_name="inventory_processed")
# drop duplicates in Title column



In [2]:
df_raw.drop_duplicates(subset=['Title'], keep='last')
df = df_raw.copy()

In [6]:
# for destination replace all with rows with just the destination so select the word between - and -
# select the word after the last - for Destination column
df = df_raw.copy()
df["Destination"] = df['Destination'].str.extract(r'-(.*)$')
df["Type"] = df['Destination'].str.extract(r'-(.*)$')
# now Destination column should only have the destination name
df["Destination"] = df['Destination'].str.extract(r'(.*)-')
# Rename Destination to Location, Text to Description
df.rename(columns={"Destination": "Location", "Text": "Description"}, inplace=True)
df["Country"] = "Malaysia"
df["Tags"] = ""
df["Duration"]=""
df["The data ingested isn't shown on the flyers."] = ""
df["Vendor ID"] = ""
df["Website Ingestion instead?"] = ""
df["Complexity Level & Error Rate"] = ""
df["Completion Rate & Accurate Text Capture Rate"] = ""
df["Remarks"] = ""
df["Ranking"] = ""
df["Item"] = ""
df = df[['Item', 'Country', 'Location', 'Type', 'Tags', 'Duration', 'Title',
    'Description', 'Vendor ID','Ranking']]



In [8]:
df

,Item,Country,Location,Type,Tags,Duration,Title,Description,Vendor ID,Ranking
0,,Malaysia,putrajaya,cruise,,,cruise tasik putrajaya,Cruise Tasik Putrajaya\nCruise beyond your im...,,
1,,Malaysia,malacca,attractions,,,menara taming sari,MENARA TAMING SARI\nMELAKA\n\nTaming Sari Tow...,,
2,,Malaysia,malacca,attractions,,,melaka river cruise,MELAKA RIVER CRUISE\nS.M.A. MELAKA\n\nwww.mel...,,
3,,Malaysia,terengganu,accommodation,,,perhentian island resort,**Perhentian Island Resort**\n\nPulau Perhent...,,
4,,Malaysia,terengganu,tour package,,,ag holidays,Pakej Aktiviti\nKem Pelajar & IPT\nSetu Adven...,,
...,...,...,...,...,...,...,...,...,...,...
142,,Malaysia,kedah,accommodation transportation,,,tropical charters langkawi,**MEGA PROMO LANGKAWI 3D2N PACKAGES**\n**NADI...,,
143,,Malaysia,kedah,excursion,,,discover langkawi treetopia 2,# PARK Map\n\n## COLOSSAL COLOSSAL FEEDING\nN...,,
144,,Malaysia,kedah,accommodation,,,mercure hotels & resorts,# Mercure\n## Langkawi Pantai Cenang\n### MAT...,,
145,,Malaysia,kedah,accommodation excursion,,,adventure travel,# HOT COUPLE PACKAGE\n## Explore Famous Activ...,,


In [13]:
from gdrive import gdrive_handler
import importlib
importlib.reload(gdrive_handler)
from gdrive.gdrive_handler import GspreadHandler
CREDENTIALS_FILE = 'smart-platform.json'
SHEET_NAME = "Master Database" 
WORKSHEET_NAME = "inventory_processed"

gspread_handler = GspreadHandler(credentials_filepath=CREDENTIALS_FILE)
def update_google_sheet(df):
    """Updates the Google Sheet with the extracted text."""
    # keep only the columns we want to update
    gspread_handler.update_cols(df, SHEET_NAME, WORKSHEET_NAME) # replace with the correct sheet name 

update_google_sheet(df)

Data appended to 'Master Database' - 'inventory_processed' (matching columns only)


In [1]:
pip install cohere

  Using cached fastavro-1.9.4-cp312-cp312-win_amd64.whl.metadata (5.7 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached tokenizers-0.15.2-cp312-none-win_amd64.whl.metadata (6.8 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/168.9 kB ? eta -:--:--
   ---------------------------------------- 168.9/168.9 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0